In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.metrics import confusion_matrix

In [50]:
def y_trans(y):
    if y > 0.05:
        return 1
    elif y > 0.15:
        return 2
    else:
        return 0
    
def cate(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [69]:
s = time.time()

data = pd.read_csv('train_data.csv',index_col = 0)
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()

data = data[(data.iloc[:,-1] < np.percentile(data.iloc[:,-1],99.96)) & (data.iloc[:,-1] < np.percentile(data.iloc[:,-1],99.96))]
#data.iloc[:,-1] = data.iloc[:,-1].map(y_trans)

x = data.iloc[:,:-1]
y = data.iloc[:,-1].to_numpy().reshape(-1,1)

In [70]:
x_scaler = MinMaxScaler().fit(x)
x = x_scaler.transform(x)

In [71]:
y_scaler = StandardScaler().fit(y)
y = y_scaler.transform(y)

In [72]:
pd.Series([x[0] for x in y]).describe()

count    7.473900e+04
mean    -3.365474e-17
std      1.000007e+00
min     -9.417703e+00
25%     -3.259593e-01
50%     -9.674484e-02
75%      2.033024e-01
max      4.456869e+01
dtype: float64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

In [ ]:
my_callback = [tf.keras.callbacks.EarlyStopping(patience = 10),]

model = Sequential()
model.add(Dense(X_train.shape[1], activation = 'relu',input_shape = [X_train.shape[1],]))
model.add(Dense(512,activation = 'relu'))
model.add(Dense(1))
model.compile(loss = 'mae',optimizer = 'sgd',
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.fit(X_train, y_train,validation_split=0.3 ,epochs=100,
          callbacks = my_callback,
          batch_size = 32, use_multiprocessing = True)

Train on 35052 samples, validate on 15023 samples
Epoch 1/100
35052/35052 [==============================] - 2s 59us/step - loss: 0.4830 - mean_absolute_error: 0.4802 - val_loss: 0.4879 - val_mean_absolute_error: 0.4828
Epoch 2/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4796 - mean_absolute_error: 0.4844 - val_loss: 0.4918 - val_mean_absolute_error: 0.4827
Epoch 3/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4775 - mean_absolute_error: 0.4834 - val_loss: 0.4856 - val_mean_absolute_error: 0.4821
Epoch 4/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4764 - mean_absolute_error: 0.4819 - val_loss: 0.4856 - val_mean_absolute_error: 0.4813
Epoch 5/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4756 - mean_absolute_error: 0.4814 - val_loss: 0.4867 - val_mean_absolute_error: 0.4808
Epoch 6/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4752 - mean_absolu

Epoch 49/100
35052/35052 [==============================] - 2s 55us/step - loss: 0.4680 - mean_absolute_error: 0.4743 - val_loss: 0.4764 - val_mean_absolute_error: 0.4742
Epoch 50/100
35052/35052 [==============================] - 2s 55us/step - loss: 0.4678 - mean_absolute_error: 0.4742 - val_loss: 0.4873 - val_mean_absolute_error: 0.4742
Epoch 51/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4678 - mean_absolute_error: 0.4742 - val_loss: 0.4891 - val_mean_absolute_error: 0.4742
Epoch 52/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4676 - mean_absolute_error: 0.4742 - val_loss: 0.4836 - val_mean_absolute_error: 0.4741
Epoch 53/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4675 - mean_absolute_error: 0.4741 - val_loss: 0.4756 - val_mean_absolute_error: 0.4741
Epoch 54/100
35052/35052 [==============================] - 2s 56us/step - loss: 0.4675 - mean_absolute_error: 0.4741 - val_loss: 0.4754 - val_me

In [ ]:
y_train_pred = model.predict(X_train)
Y_pred = model.predict(X_test)

#plt.scatter(y_scaler.inverse_transform(y_train),y_scaler.inverse_transform(y_train_pred),c = 'green')
#plt.scatter(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(Y_pred),c = 'blue')
#plt.plot(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(y_test),color = 'r')
plt.scatter(y_test, Y_pred,c = 'red')
plt.scatter(y_train, y_train_pred, c = 'green')
plt.show()

In [ ]:
pd.Series([x[0] for x in Y_pred]).describe()

In [ ]:
model.summary()